<a href="https://colab.research.google.com/github/Areej17-01/Generative-AI-LLMs-and-Transformers./blob/main/7_gpt_2_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch]
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset

In [ ]:
#dataset link : https://huggingface.co/datasets/hakurei/open-instruct-v1
dataset = load_dataset("hakurei/open-instruct-v1", split='train')
dataset.to_pandas().sample(20)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

instruct_data.json:   0%|          | 0.00/104M [00:00<?, ?B/s]

additional_data.json:   0%|          | 0.00/19.7M [00:00<?, ?B/s]

alpaca_data.json:   0%|          | 0.00/22.7M [00:00<?, ?B/s]

...

self_instruct.json:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

sharegpt_data.json:   0%|          | 0.00/109M [00:00<?, ?B/s]

synthetic_instruct.json:   0%|          | 0.00/19.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/498813 [00:00<?, ? examples/s]

,output,input,instruction
93835,I would love to have super powers. I could fly...,,Write a paragraph describing what it would be ...
343353,I solved this problem by writing a function th...,,Describe a problem that you solved recently.\n...
422454,"Sure, I can help you with that. Here's a basic...",,set up NDA for a Full service Digital Agency i...
419642,This error is caused by a `BadTokenException` ...,,java.lang.RuntimeException: Error receiving br...
90637,"[""fox"", ""quick"", ""the"", ""brown""]",,Sort words by length (ascending or descending)...
165344,Some recommended books on advanced math topics...,,I need to find books on advanced math topics. ...
484518,Recipe 1: Sesame Ginger Salmon with Rice and V...,,Can you provide recipes for me to cook healthy...
215607,"Five fire pokemon are Rapidash, Magmar, Vulpix...",,Name five fire Pokemon.
305249,A time management device that helps people sta...,,Design a product to help people manage their time
366020,You can go to [City] for 3 weeks.,,I am planning to go on vacation for 3 weeks in...


In [ ]:
def preprocess(example):
    example['prompt'] = f"{example['instruction']} {example['input']} {example['output']}"

    return example

def tokenize_datasets(dataset):
    tokenized_dataset = dataset.map(lambda example: tokenizer(example['prompt'], truncation=True, max_length=128), batched=True, remove_columns=['prompt'])

    return tokenized_dataset


In [ ]:
dataset = dataset.map(preprocess, remove_columns=['instruction', 'input', 'output'])
dataset =  dataset.shuffle(42).select(range(10000)).train_test_split(test_size=0.1, seed=42)

Map:   0%|          | 0/498813 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['prompt'],
        num_rows: 1000
    })
})

In [ ]:

dataset['train'][:15]

{'prompt': ['Rewrite the following sentence using the hypernym of the underlined word: Kevin loves to watch YouTube videos. Kevin loves to watch videos.',
  "What can I do to reduce waste and be more eco-friendly.  One way to reduce waste and be more eco-friendly is to reduce your consumption of single-use plastics and opt for reusable products instead. Additionally, try to buy only as much food and other items as you need so that you don't end up throwing away unused products. You can also recycle whenever possible, reuse materials and items that you already have, and find alternative uses for items that cannot be recycled. Finally, consider growing your own fruits and vegetables and composting your food scraps to reduce your carbon footprint.",
  'Create a program in C++ that calculates the absolute difference between two numbers.\nnum1 = 50;\nnum2 = 40;  #include <iostream>\nusing namespace std;\n\nint main() \n{\n  int num1 = 50;\n  int num2 = 40;\n  int absDiff = abs(num1 - num2);

In [ ]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['prompt'],
        num_rows: 1000
    })
})

In [ ]:
#we do casual training of decoder models

In [ ]:

MODEL_NAME = "microsoft/DialoGPT-small"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

train_dataset = tokenize_datasets(train_dataset)
test_dataset = tokenize_datasets(test_dataset)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
train_dataset[0]

{'input_ids': [30003,
  6525,
  262,
  1708,
  6827,
  1262,
  262,
  8718,
  3281,
  76,
  286,
  262,
  739,
  10837,
  1573,
  25,
  7939,
  10408,
  284,
  2342,
  7444,
  5861,
  13,
  7939,
  10408,
  284,
  2342,
  5861,
  13],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

traing_args = TrainingArguments(
    output_dir="models/diablo_gpt",
    num_train_epochs=1,
    per_device_train_batch_size=8,  # Reduced batch size
    per_device_eval_batch_size=8,  # Match evaluation batch size
    gradient_accumulation_steps=4,  # Simulates effective batch size of 32
    fp16=True,  # Mixed precision
    save_steps=500,  # Save less frequently
    logging_steps=100,  # Log less frequently
    save_total_limit=2,  # Limit the number of checkpoints saved
    evaluation_strategy="steps",  # Evaluate periodically
    eval_steps=500,  # Evaluate every 500 steps
    load_best_model_at_end=True,  # Use best model for evaluation
)


trainer = Trainer(model=model,
                    args=traing_args,
                    train_dataset=train_dataset,
                    eval_dataset=test_dataset,
                    data_collator=data_collator)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 151


...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss


In [ ]:
!pip install huggingface_hub


In [ ]:
from huggingface_hub import login

login("your api key")


In [ ]:
from huggingface_hub import create_repo

# Create a repository
repo_name = "diabloGPT_open-instruct-small"
create_repo(repo_id=repo_name, private=False)  # Set private=True if you want it to be private


In [ ]:
from huggingface_hub import upload_folder


model_dir = "/content/models/diablo_gpt/checkpoint-14"

upload_folder(
    folder_path=model_dir,
    repo_id="path to trained model",
    commit_message="Initial commit for DiabloGPT"
)


In [ ]:
from transformers import AutoConfig

# Replace "gpt2" with your base model type if it's different
config = AutoConfig.from_pretrained("gpt2")

# Customize if needed
config.save_pretrained("diabloGPT_open-instruct-small")


In [ ]:
config

In [ ]:
model = AutoModelForCausalLM.from_pretrained("path to trained model")


In [ ]:
def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors='pt', padding=True)
    outputs = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=64,
    min_length=10,
    num_beams=5,
    temperature=1.0,
    pad_token_id=tokenizer.eos_token_id
    )

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated


In [ ]:
generate_text("What's the best way to cook chiken breast?")

In [ ]:
generate_text("I need a place to go for this summer vacation, what locations would you recommend")


In [ ]:
generate_text("What's the fastest route from NY City to Boston?")


In [ ]:
generate_text("hi")


In [ ]:
generate_text("who are you")
